In [11]:
%matplotlib inline

In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [1]:
# Standard library
import concurrent.futures
import csv
import datetime
import functools
import glob
import gzip
import importlib
import itertools
import json
import logging
import multiprocessing as mp
import os
import os.path as op
import pathlib
import pickle
import re
import shlex
import shutil
import string
import subprocess
import sys
import tempfile
import time
import urllib
from collections import Counter, defaultdict, OrderedDict, namedtuple
from pathlib import Path
from pprint import pprint
from textwrap import dedent

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import scipy as sp
import seaborn as sns
import sqlalchemy as sa
import tqdm
import wurlitzer
import yaml

from IPython.display import display, HTML, Math

In [14]:
import kmbio.PDB

import kmtools
import kmtools.df_tools
import kmtools.sequence_tools
import kmtools.structure_tools

import elaspic
import elaspic.elaspic_predictor
import elaspic.structure_tools

import jobsubmitter

import odbo

print2 = kmtools.df_tools.print2

In [15]:
# Create logger
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create STDERR handler
handler = logging.StreamHandler(sys.stderr)
handler.setLevel(logging.DEBUG)

# Create formatter and add it to the handler
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)

# Set STDERR handler as the only handler 
logger.handlers = [handler]

In [16]:
class LoggingContext(object):
    def __init__(self, logger, level=None, handler=None, close=True):
        self.logger = logger
        self.level = level
        self.handler = handler
        self.close = close

    def __enter__(self):
        if self.level is not None:
            self.old_level = self.logger.level
            self.logger.setLevel(self.level)
        if self.handler:
            self.logger.addHandler(self.handler)

    def __exit__(self, et, ev, tb):
        if self.level is not None:
            self.logger.setLevel(self.old_level)
        if self.handler:
            self.logger.removeHandler(self.handler)
        if self.handler and self.close:
            self.handler.close()
        # implicit return of None => don't swallow exceptions

In [17]:
try:
    import local
except ImportError:
    pass

In [18]:
try:
    fin = open('../.gitlab-ci.yml')
except FileNotFoundError:
    CONFIG = dict()
else:
    CONFIG = yaml.load(fin)['variables']
    for key, value in CONFIG.items():
        logger.info("Setting the %s environment variable.", key)
        os.environ[key] = str(value)  # environment variables can't be integers or anything else
finally:
    fin.close()

Setting the PYTHON_VERSION environment variable.
Setting the SPARK_MASTER environment variable.
Setting the SPARK_ARGS environment variable.
Setting the DB_TYPE environment variable.
Setting the DB_PORT environment variable.


In [19]:
os.environ['DB_SCHEMA'] = op.basename(op.dirname(os.getcwd()))
os.environ['DB_PORT'] = str(CONFIG['DB_PORT'])
os.environ['DB_TEMPDIR'] = op.join(tempfile.gettempdir(), op.basename(op.dirname(os.getcwd())))

In [20]:
# Display options
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)
pd.set_option("display.max_colwidth", 120)
pd.set_option('mode.chained_assignment', None)
pd.set_option('io.hdf.default_format','table')

sns.set_style('whitegrid')
sns.set_context('notebook', font_scale=2)

In [21]:
print(datetime.datetime.now())

2017-11-21 18:30:41.795555


In [ ]:
_IMPORTS_LOADED = True